<a href="https://colab.research.google.com/github/gibiee/Measure_BoneAge/blob/master/%EB%BC%88%20%EB%82%98%EC%9D%B4%20%EC%B8%A1%EC%A0%95(Male%20%EC%A0%84%EC%9A%A9).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

male_model = tf.keras.applications.InceptionV3(weights=None, include_top=True, input_shape=(224,224,1), classes=1, classifier_activation=None)
female_model = tf.keras.applications.InceptionV3(weights=None, include_top=True, input_shape=(224,224,1), classes=1, classifier_activation=None)

In [ ]:
male_model.compile(loss='mae', optimizer='adam')
female_model.compile(loss='mae', optimizer='adam')

In [ ]:
import pandas as pd

csv_train=pd.read_csv('./boneage-training-dataset.csv')

for i in range(len(csv_train['id'])):
  csv_train.loc[i,"id"] = f"{csv_train['id'][i]}.png"

printf("\n******************** boneage-training-dataset ****************************\n")

In [ ]:
male_df = csv_train.loc[csv_train.male == True]
female_df = csv_train.loc[csv_train.male == False]

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2) # random으로 validation 뽑음

train_generator_m = datagen.flow_from_dataframe(male_df, directory='./boneage-training-dataset', x_col='id', y_col='boneage',
                                              target_size=(224,224), color_mode='grayscale',
                                              class_mode='raw', subset='training')
valid_generator_m = datagen.flow_from_dataframe(male_df, directory='./boneage-training-dataset', x_col='id', y_col='boneage',
                                              target_size=(224,224), color_mode='grayscale', 
                                              class_mode='raw', subset='validation', shuffle=False)


train_generator_f = datagen.flow_from_dataframe(female_df, directory='./boneage-training-dataset', x_col='id', y_col='boneage',
                                              target_size=(224,224), color_mode='grayscale',
                                              class_mode='raw', subset='training')
valid_generator_f = datagen.flow_from_dataframe(female_df, directory='./boneage-training-dataset', x_col='id', y_col='boneage',
                                              target_size=(224,224), color_mode='grayscale', 
                                              class_mode='raw', subset='validation', shuffle=False)


printf("\n******************** MAKE Gegerator ****************************\n")


In [ ]:

# Identity Block
# class IdentityBlock(tf.keras.Model):
#     def __init__(self, filters, kernel_size):
#         super(IdentityBlock, self).__init__(name='')
 
#         self.conv1 = tf.keras.layers.Conv2D(filters, kernel_size, padding='same')
#         self.bn1 = tf.keras.layers.BatchNormalization()
 
#         self.conv2 = tf.keras.layers.Conv2D(filters, kernel_size, padding='same')
#         self.bn2 = tf.keras.layers.BatchNormalization()
 
#         self.relu = tf.keras.layers.Activation('relu')
#         self.add = tf.keras.layers.Add()
    
#     def call(self, inputs):
#         x = self.conv1(inputs)
#         x = self.bn1(x)
#         x = self.relu(x)
 
#         x = self.conv2(x)
#         x = self.bn2(x)
 
#         x = self.add([x, inputs])
#         x = self.relu(x)
 
#         return x

# class ResNet(tf.keras.Model):
#     def __init__(self, num_classes):
#         super(ResNet, self).__init__()
#         self.conv = tf.keras.layers.Conv2D(64, 7, padding='same')
#         self.bn = tf.keras.layers.BatchNormalization()
#         self.relu = tf.keras.layers.Activation('relu')
#         self.max_pool = tf.keras.layers.MaxPool2D((3, 3))
#         self.id1a = IdentityBlock(64, 3)
#         self.id1b = IdentityBlock(64, 3)
#         self.global_pool = tf.keras.layers.GlobalAveragePooling2D()
#         self.classifier = tf.keras.layers.Dense(num_classes, activation='softmax')
    
#     def call(self, inputs):
#         x = self.conv(inputs)
#         x = self.bn(x)
#         x = self.relu(x)
#         x = self.max_pool(x)
 
#         x = self.id1a(x)
#         x = self.id1b(x)
 
#         x = self.global_pool(x)
#         return self.classifier(x)

# resnet = resnet(10)
# resnet.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])

# resnet.fit(train_generator, steps_per_epoch=100, epochs=1, validation_data=valid_generator, validation_steps=800)
# resnet.save("/content/drive/Shareddrives/growthPrediction/machineLearning/check_point_4")
 
 


In [ ]:
checkpoint_m = tf.keras.callbacks.ModelCheckpoint(monitor='val_loss', save_best_only=True, filepath="./check_point_m")
checkpoint_f = tf.keras.callbacks.ModelCheckpoint(monitor='val_loss', save_best_only=True, filepath="./check_point_f")

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

history_m = male_model.fit_generator(train_generator_m, validation_data=valid_generator_m, epochs=1, callbacks=[checkpoint_m, early_stopping])
history_f = female_model.fit_generator(train_generator_f, validation_data=valid_generator_f, epochs=1, callbacks=[checkpoint_f, early_stopping])

# male_model.fit(train_generator_m, steps_per_epoch=100, epochs=1, validation_data=valid_generator_m, validation_steps=800)
# female_model.fit(train_generator_f, steps_per_epoch=100, epochs=1, validation_data=valid_generator_f, validation_steps=800)

# male_model.save(./check_point_m)
# female_model.save(./check_point_f)
printf("\n******************** FINISH save model ****************************\n")
